In [1]:
import os
import sys
import torch
import socket
import struct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

import random
import spinup
import core_FPGA as core

from utils import weights_creation, create_input_tensor


In [2]:
N_EPISODES = 20

In [3]:
hidden_sizes = [core.N_HIDDEN_1, core.N_HIDDEN_2]

In [4]:
SAMPLE_RATE = 100000  # 100 kHz
TIME_STEP = 1 / SAMPLE_RATE

MAX_VAL = 32767
MAX_V_DAC = 5
MAX_V_ADC = 25

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [5]:
import torch
import numpy as np
import os
import struct
import socket

def request_and_receive_data(server_ip, server_port, experiment_folder):
  # Set up the socket connection
  client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  client.settimeout(60 * 60)  # Set the timeout for blocking socket operations, 1h
  client.connect((server_ip, server_port))

  # Instantiate the digital twin 
  model = core.Digital_twin(core.N_INPUT, 1, hidden_sizes, device).to(device)

  # Have to run inference once - create random data
  in_float = create_input_tensor(core.N_INPUT, device)
  
  model.eval()
  
  out_brevitas = model(in_float)

  scale = float(out_brevitas.scale[0, 0])  

  for ep in range(N_EPISODES):
      weights_list, random_numbers = weights_creation(model, scale, ep)
      
      print(f"Ready for Run {ep}/{N_EPISODES-1}:")
      
      # Serialize and send the weights list; here, each weight is packed as a single byte, reflecting the int8 data type.
      packed_weights = b''.join(struct.pack('<b', weight) for weight in weights_list)
      client.send(packed_weights) # Send it all 

      print("Done streaming weights.")
      
      obs, actions = core.receive_data_episode(client, core.N_STEPS * core.N_INPUT, core.N_STEPS)

      # Save the data
      save_data(obs, actions, ep, experiment_folder)

  # Save the model
  save_model(model, experiment_folder)

  # Close the socket connection
  client.close()

def save_data(obs, actions, episode, experiment_folder):
  data_folder = os.path.join(experiment_folder, 'data')
  os.makedirs(data_folder, exist_ok=True)
  
  np.save(os.path.join(data_folder, f'obs_episode_{episode}.npy'), obs)
  np.save(os.path.join(data_folder, f'actions_episode_{episode}.npy'), actions)

def save_model(model, experiment_folder):
  model_folder = os.path.join(experiment_folder, 'model')
  os.makedirs(model_folder, exist_ok=True)
  
  torch.save(model.state_dict(), os.path.join(model_folder, 'model.pth'))


In [6]:
experiment_folder = 'new_no_over_scope_data'
server_ip = '192.168.1.10'
server_port = 7

errors = request_and_receive_data(server_ip, server_port, experiment_folder)

C:\Users\giuli\Anaconda3\envs\brevitas_env\Lib\site-packages\torch\_tensor.py:1413: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/core/TensorImpl.h:1928.)
  return super().rename(names)


Sent Scale: 3.9910099964429735e-13
Ready for Run 0/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 1/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 2/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 3/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 4/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 5/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 6/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 7/19:
Done streaming weights.
Number of received actions: 324
Sent Scale: 3.9910099964429735e-13
Ready for Run 8/19:
Done streaming weights.
Number of received actions: 324
S

In [11]:
def load_model(experiment_folder):
  model_path = os.path.join(experiment_folder, 'model', 'model.pth')
  
  # Instantiate the model with the same architecture as when it was saved
  model = core.Digital_twin(core.N_INPUT, 1, hidden_sizes, device).to(device)
  
  # Load the state dict
  state_dict = torch.load(model_path, map_location=device)
  
  # Try to load the state dict, ignoring mismatched keys
  model.load_state_dict(state_dict, strict=False)
  
  model.eval()
  return model

In [12]:
experiment_folder

'new_no_over_scope_data'

In [13]:
model = load_model(experiment_folder)

C:\Users\giuli\AppData\Local\Temp\ipykernel_21924\1678710607.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)
